In [4]:
from utils import *

In [5]:
image = cv2.imread("Data3/one/image "+str(1)+".jpg")

In [6]:
image2 = np.random.randint(0,250,(700,393,3)).astype("int")

In [7]:
cv2.imshow("image random",image2)
cv2.waitKey(10000)

error: OpenCV(4.5.5) /io/opencv/modules/highgui/src/precomp.hpp:155: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


images2 = np.random.rand()

In [17]:
one = []
zero = []

for i in range (1,236):

    one.append(cv2.imread("Data3/one/image "+str(i)+".jpg"))
    zero.append(cv2.imread("Data3/zero/image "+str(i)+".jpg"))

One = np.array(one)
Zero = np.array(zero)
oneA = One.reshape(235,-1)
zeroA = Zero.reshape(235,-1)

ones = np.ones((235,1))
zeros = np.zeros((235,1))

dataone = np.hstack([oneA,ones])
datazero = np.hstack([zeroA,zeros])

data = np.vstack([dataone,datazero])/255
np.random.shuffle(data)

In [27]:
X = data[:,:-1].reshape(-1,128,128,3)
Y = data[:,-1]
Y = np.array(tf.one_hot(Y,2)).reshape(-1,2)

In [31]:
model = Sequential()

model.add(InputLayer((128,128,3)))
model.add(Convolution2D(32,(4,4),padding="same",activation=relu))
model.add(Convolution2D(64,(4,4),padding="same",activation=relu))
model.add(Convolution2D(128,(4,4),padding="same",activation=relu))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Convolution2D(128,(4,4),padding="same",activation=relu))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Convolution2D(256,(4,4),padding="same",activation=relu))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(1000,activation=relu))
model.add(Dropout(0.2))
model.add(Dense(100,activation=relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=relu))
model.add(Dropout(0.2))
model.add(Dense(2,activation=relu))
model.compile(loss=mean_absolute_error,optimizer=Adam(),metrics=Accuracy())
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 128, 128, 32)      1568      
                                                                 
 conv2d_31 (Conv2D)          (None, 128, 128, 64)      32832     
                                                                 
 conv2d_32 (Conv2D)          (None, 128, 128, 128)     131200    
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 64, 64, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 64, 64, 128)       262272    
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 32, 32, 128)      0         
 g2D)                                                 

In [35]:
model.fit(X,Y,epochs=20)

Epoch 1/20
 4/15 [=======>......................] - ETA: 39s - loss: 0.1290 - accuracy: 0.4883

In [34]:
res = model.predict(X)

In [5]:
from utils import *
from count_fingers import *
from predict import *
from run_avg import *
from segment import *
from global_vars import *

def video():

    aWeight = 0.1

    bg = None;

    camera = cv2.VideoCapture(0)

    top, right, bottom, left = 10, 350, 225, 590

    prevfin = 0

    fingers = 0

    num_frames = 0

    fr = 0

    num = 1

    finger_disp = 0

    fingerC = 0

    while(True):

        (grabbed, frame) = camera.read()

        frame = imutils.resize(frame, width=700)

        frame = cv2.flip(frame, 1)

        clone = frame.copy()

        (height, width) = frame.shape[:2]

        roi = frame[top:bottom, right:left]

        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 50:
            
            bg = RunAvg(gray, aWeight,bg)

        else:

            if (num_frames == 50):
                print(''' 

                Background Readed
                
                We are good to Go !!!
                
                ''')
            
            hand = segment(gray,bg)

            if hand is not None:
                
                (thresholded, segmented) = hand

                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                
                fingers,chull = count(thresholded,segmented)      

                if num % 10 == 0:

                    finger_disp = fingerC

                    fingerC = 0

                else:
                    fingerC += fingers

                    num += 1

                res = predict(thresholded)
                
                cv2.putText(clone, str(int(fingers)), (70, 85), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

                cv2.imshow("Thesholded", thresholded)
                
                cv2.putText(clone,res, (70, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

                # if (num_frames%2 == 0):

                #     if (res == "front" or res == "back"):

                #         x.front_back(res)
                    
                #     if (res == "left" or res == "right"):

                        # x.left_right(res)
                
        prevfin = fingers

        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)
        
        num_frames += 1

        cv2.imshow("Video Feed", clone)
        
        keypress = cv2.waitKey(1) & 0xFF

        if num_frames == 100000:

            break



